# Hello RRSR!

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from scipy.stats import norm
import itertools
from collections import defaultdict

In [2]:
# 1. 加载Iris数据集并划分为训练集和测试集
iris = load_iris()
X = iris.data  # 四个属性
y = iris.target  # 三个类 (0, 1, 2)
num_classes = len(np.unique(iris.target))
num_attributes = iris.data.shape[1]
# 将数据集划分为训练集和测试集，乱序
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

## RPS generation method

### Step 1: 
Establish Gaussian discriminant model (GDM), and then construct membership vector based on the GDM.

In [3]:
# 2. 计算每个类中每个属性的 mean value and standard deviation (无偏估计)
mean_std_by_class = []
for class_label in np.unique(y_train):
    X_class = X_train[y_train == class_label]
    mean_std = [(np.mean(X_class[:, i]), np.std(X_class[:, i], ddof=1)) for i in range(X_class.shape[1])]
    mean_std_by_class.append(mean_std)

mean_std_by_class = np.array(mean_std_by_class)
print("每个类中每个属性的均值和标准差:\n", mean_std_by_class)
print("Shape of mean_std_by_class:\n", mean_std_by_class.shape)

每个类中每个属性的均值和标准差:
 [[[4.99       0.3564785 ]
  [3.4525     0.39547926]
  [1.45       0.18397324]
  [0.245      0.10609623]]

 [[5.9195122  0.54231887]
  [2.77073171 0.32034661]
  [4.24146341 0.4811318 ]
  [1.32195122 0.20556288]]

 [[6.53333333 0.65386838]
  [2.96666667 0.31898963]
  [5.52051282 0.5415278 ]
  [2.         0.2901905 ]]]
Shape of mean_std_by_class:
 (3, 4, 2)


In [4]:
# 3. 为每个类和每个属性建立高斯分布函数，并对测试集中随机选取的一个样本进行预测

# 保存下(3,4)个Gaussian distribution函数
# 创建一个(3,4)的函数数组，用来存储每个类中每个属性的高斯分布函数
gaussian_functions = np.empty((3, 4), dtype=object)

# 初始化并保存高斯分布函数
for class_label in range(num_classes):
    for i in range(num_attributes):  # 四个属性
        mean, std = mean_std_by_class[class_label, i]
        # 保存高斯分布函数
        gaussian_functions[class_label, i] = norm(loc=mean, scale=std)

# 随机选择一个测试集中的样本
test_sample = X_test[np.random.randint(0, len(X_test))]

# 计算该测试样本在每个类中每个属性的高斯分布结果
gaussian_results = []
for class_label in range(num_classes):
    class_results = []
    for i in range(num_attributes):  # 四个属性
        # 调用保存的高斯分布函数，计算概率密度值
        pdf_value = gaussian_functions[class_label, i].pdf(test_sample[i])
        class_results.append(pdf_value)
    gaussian_results.append(class_results)

gaussian_results = np.array(gaussian_results)
print("\n测试集中选取的样本:", test_sample)
print("\n每个类中每个属性的高斯分布函数值:\n", gaussian_results)


测试集中选取的样本: [6.8 2.8 4.8 1.4]

每个类中每个属性的高斯分布函数值:
 [[2.82308256e-06 2.58628797e-01 2.16673472e-72 6.92713895e-26]
 [1.96909178e-01 1.24015881e+00 4.22677240e-01 1.80576639e+00]
 [5.61439257e-01 1.09107548e+00 3.04003048e-01 1.62152041e-01]]


### Step 2: 
Perform weight analysis for the test sample.

In [5]:
column_sums = np.sum(gaussian_results, axis=0)
normalized_results = gaussian_results / column_sums
print("\n每个属性针对所有类的归一化后的MV (归一化后的高斯分布值):\n", normalized_results)


每个属性针对所有类的归一化后的MV (归一化后的高斯分布值):
 [[3.72265824e-06 9.98619571e-02 2.98168914e-72 3.52003358e-26]
 [2.59654318e-01 4.78851109e-01 5.81655024e-01 9.17602255e-01]
 [7.40341960e-01 4.21286934e-01 4.18344976e-01 8.23977452e-02]]


In [6]:
# 对归一化后的MV（normalized membership vector）进行降序排序，并保留原始顺序的索引
sorted_indices = np.argsort(-normalized_results, axis=0)  # 降序排序，使用负号实现降序
sorted_nmv = np.take_along_axis(normalized_results, sorted_indices, axis=0)  # 按照索引排序后的值
sorted_gaussian_functions = np.take_along_axis(gaussian_functions, sorted_indices, axis=0) # 按照索引排序后的GDM

# 打印结果
print("\n归一化后的MV降序排序的结果:\n", sorted_nmv)
print("\n每个元素排序前的原始类索引:\n", sorted_indices)



归一化后的MV降序排序的结果:
 [[7.40341960e-01 4.78851109e-01 5.81655024e-01 9.17602255e-01]
 [2.59654318e-01 4.21286934e-01 4.18344976e-01 8.23977452e-02]
 [3.72265824e-06 9.98619571e-02 2.98168914e-72 3.52003358e-26]]

每个元素排序前的原始类索引:
 [[2 1 1 1]
 [1 2 2 2]
 [0 0 0 0]]


In [7]:
x_mean_ord = np.empty((3, 4))
std_ord = np.empty((3, 4))


# mean_std_by_class 的 shape 是 (3, 4, 2)，索引 [class, attribute, 0] 获取均值，索引 [class, attribute, 1] 获取标准差
for attr_idx in range(num_attributes):  # 对每个属性进行操作
    for class_idx in range(num_classes):  # 对每个类进行操作
        sorted_class_idx = sorted_indices[class_idx, attr_idx]  # 获取排序后的类索引
        x_mean_ord[class_idx, attr_idx] = mean_std_by_class[sorted_class_idx, attr_idx, 0]  # 获取排序后的均值
        std_ord[class_idx, attr_idx] = mean_std_by_class[sorted_class_idx, attr_idx, 1]  # 获取排序后的标准差

print("\n排序后的 x_mean_ord:\n", x_mean_ord)
print("\n排序后的 std_ord:\n", std_ord)


排序后的 x_mean_ord:
 [[6.53333333 2.77073171 4.24146341 1.32195122]
 [5.9195122  2.96666667 5.52051282 2.        ]
 [4.99       3.4525     1.45       0.245     ]]

排序后的 std_ord:
 [[0.65386838 0.32034661 0.4811318  0.20556288]
 [0.54231887 0.31898963 0.5415278  0.2901905 ]
 [0.3564785  0.39547926 0.18397324 0.10609623]]


In [8]:
supporting_degree = np.exp(-np.abs(test_sample - x_mean_ord))

print("\nSupporting degree (支持度):\n", supporting_degree)


Supporting degree (支持度):
 [[0.76592834 0.97115588 0.57204559 0.92491931]
 [0.41458063 0.84648172 0.4865027  0.54881164]
 [0.16365414 0.52074229 0.03508435 0.31505754]]


In [9]:
# 生成所有按顺序选择的排列组合
def get_ordered_permutations(num_classes):
    result = []
    # 逐步增加元素数量
    for i in range(1, num_classes + 1):
        # 生成i个元素的全排列
        result.extend(itertools.permutations(range(i), i))
    return result

# 获取按顺序选择的排列组合
all_combinations = get_ordered_permutations(num_classes)
all_combinations

[(0,),
 (0, 1),
 (1, 0),
 (0, 1, 2),
 (0, 2, 1),
 (1, 0, 2),
 (1, 2, 0),
 (2, 0, 1),
 (2, 1, 0)]

In [10]:
# supporting_degree: 形状为 (3, 4) 的支持度矩阵 (3个类，4个属性)
# num_classes: 类别的数量 (3)
# num_attributes: 属性的数量 (4)


In [11]:
# 初始化权重矩阵 weight_matrix
num_combinations = len(all_combinations)  # 所有按顺序排列组合的数量 (应该是9)
weight_matrix = np.zeros((num_combinations, num_attributes))  # (9, 4)


# 对每个属性计算权重
for attr_idx in range(num_attributes):
    s = supporting_degree[:, attr_idx]  # 取出该属性对应的支持度 (3,)
    
    # 遍历每个组合，计算 w(i1...iu...iq)
    for comb_idx, combination in enumerate(all_combinations):
        q = len(combination)  # 该组合的长度
        weight = 1.0  # 初始化权重
        
        # 根据公式 (19) 计算权重
        for u in range(q):
            
            i_u = combination[u]  # 当前排列项 i_u
            numerator = s[i_u]  # 分子支持度
            denominator_sum = np.sum(s[list(combination[u:])])  # 分母，从 u 到 q 的支持度和
            weight *= numerator / denominator_sum  # 按公式累乘
        
        # 将计算好的权重保存到 weight_matrix
        weight_matrix[comb_idx, attr_idx] = weight

# 输出权重矩阵
print("\n权重矩阵 (Weight matrix):\n", weight_matrix)


权重矩阵 (Weight matrix):
 [[1.         1.         1.         1.        ]
 [0.64881196 0.53429566 0.54040576 0.62760391]
 [0.35118804 0.46570434 0.45959424 0.37239609]
 [0.40854601 0.25712943 0.48788518 0.32848852]
 [0.16127199 0.15818199 0.03518405 0.18857614]
 [0.25413077 0.23564177 0.41914354 0.22885194]
 [0.05429953 0.12635318 0.02570666 0.0779544 ]
 [0.07899396 0.11898424 0.01733652 0.11053925]
 [0.04275774 0.10370939 0.01474404 0.06558975]]


### Step 3: 
Construct weighted PMF based on weight vector and ONMV, and then generate weighted RPS.

In [12]:
# 计算 weighted PMF
def calculate_weighted_pmf(weight_matrix, sorted_nmv):
    num_combinations, num_attributes = weight_matrix.shape
    num_classes = sorted_nmv.shape[0]  # 获取类的数量（classes）
    
    # 获取排列组合
    all_combinations = get_ordered_permutations(num_classes)
    
    # 初始化 weighted_pmf 矩阵
    weighted_pmf = np.zeros_like(weight_matrix)
    
    # 记录当前组合数对应的起始位置
    current_row = 0
    
    # 遍历组合大小 i（从 1 到 num_classes）
    for i in range(1, num_classes + 1):
        num_permutations = len(list(itertools.permutations(range(i), i)))  # 当前大小的排列组合数量
        
        # 遍历每个属性 j
        for j in range(num_attributes):
            # 对于当前大小 i 的排列组合，使用 sorted_nmv[i-1, j]
            weighted_pmf[current_row:current_row + num_permutations, j] = (
                weight_matrix[current_row:current_row + num_permutations, j] * sorted_nmv[i-1, j]
            )
        
        # 更新起始行
        current_row += num_permutations
    
    return weighted_pmf


用于测试实现正确

In [13]:
# 示例
# 假设 weight_matrix 是一个 (9, 4) 的矩阵，sorted_nmv 是一个 (3, 4) 的矩阵
# sorted_nmv 示例数据
sorted_nmv_FORTEST = np.array([
    [0.5, 0.6, 0.7, 0.6189],
    [0.4, 0.5, 0.6, 0.3811],
    [0.3, 0.4, 0.5, 1.57e-32]
])

# weight_matrix 示例数据
weight_FORTEST = np.array([
    [1.0, 0.9, 0.8, 1],
    [0.6, 0.5, 0.4, 0.5374],
    [0.2, 0.1, 0.3, 0.4626],
    [0.5, 0.7, 0.6, 0.0828],
    [0.4, 0.6, 0.5, 0.0713],
    [0.3, 0.2, 0.4, 0.1284],
    [0.2, 0.3, 0.5, 0.3262],
    [0.1, 0.2, 0.3, 0.0990],
    [0.2, 0.1, 0.4, 0.2923]
])

# 调用函数计算 weighted PMF
weighted_FORTEST = calculate_weighted_pmf(weight_FORTEST, sorted_nmv_FORTEST)
print("\n测试用 PMF:\n", weighted_FORTEST)


测试用 PMF:
 [[5.0000000e-01 5.4000000e-01 5.6000000e-01 6.1890000e-01]
 [2.4000000e-01 2.5000000e-01 2.4000000e-01 2.0480314e-01]
 [8.0000000e-02 5.0000000e-02 1.8000000e-01 1.7629686e-01]
 [1.5000000e-01 2.8000000e-01 3.0000000e-01 1.2999600e-33]
 [1.2000000e-01 2.4000000e-01 2.5000000e-01 1.1194100e-33]
 [9.0000000e-02 8.0000000e-02 2.0000000e-01 2.0158800e-33]
 [6.0000000e-02 1.2000000e-01 2.5000000e-01 5.1213400e-33]
 [3.0000000e-02 8.0000000e-02 1.5000000e-01 1.5543000e-33]
 [6.0000000e-02 4.0000000e-02 2.0000000e-01 4.5891100e-33]]


In [14]:
weighted_pmf =  calculate_weighted_pmf(weight_matrix, sorted_nmv)
print("\nWeighted PMF:\n", weighted_pmf)


Weighted PMF:
 [[7.40341960e-01 4.78851109e-01 5.81655024e-01 9.17602255e-01]
 [1.68466827e-01 2.25091779e-01 2.26076033e-01 5.17131473e-02]
 [9.11874904e-02 1.96195155e-01 1.92268943e-01 3.06845979e-02]
 [1.52087717e-06 2.56774478e-02 1.45472195e-72 1.15629060e-26]
 [6.00360520e-07 1.57963635e-02 1.04907906e-73 6.63794362e-27]
 [9.46041991e-07 2.35316485e-02 1.24975575e-72 8.05566501e-27]
 [2.02138604e-07 1.26178762e-02 7.66492634e-74 2.74402097e-27]
 [2.94067503e-07 1.18819987e-02 5.16921205e-74 3.89101884e-27]
 [1.59172450e-07 1.03566224e-02 4.39621540e-74 2.30878132e-27]]


In [15]:
def get_acc_permutations(num):
    all_combinations_ = []
    for r in range(1, num + 1):  
        permutations_ = list(itertools.permutations(range(num), r))
        all_combinations_.extend(permutations_)

    return len(all_combinations_)
assert get_acc_permutations(3) == 15

In [16]:
RPS_w = []
for j in range(num_attributes):
    RPS_w_j = set()
    
    thetas = sorted_indices[:, j]
    weighted_pmf_j = weighted_pmf[:, j]
    
    for idx, combination in enumerate(all_combinations):
        A = thetas[list(combination)]
        M_A = weighted_pmf_j[idx]
        A = tuple((A))
        RPS_w_j.add((A, M_A))
    
    RPS_w.append(RPS_w_j)

RPS_w

[{((0, 1, 2), 1.5917244981330598e-07),
  ((0, 2, 1), 2.9406750290725125e-07),
  ((1, 0, 2), 2.0213860437974062e-07),
  ((1, 2), 0.09118749040651578),
  ((1, 2, 0), 9.460419907293513e-07),
  ((2,), 0.7403419597193839),
  ((2, 0, 1), 6.003605197431245e-07),
  ((2, 1), 0.16846682721585785),
  ((2, 1, 0), 1.5208771748850276e-06)},
 {((0, 1, 2), 0.01188199867148949),
  ((0, 2, 1), 0.01035662243762494),
  ((1,), 0.47885110874073056),
  ((1, 0, 2), 0.015796363493213478),
  ((1, 2), 0.22509177919736081),
  ((1, 2, 0), 0.025677447776139048),
  ((2, 0, 1), 0.012617876180362793),
  ((2, 1), 0.19619515499138454),
  ((2, 1, 0), 0.02353164851169426)},
 {((0, 1, 2), 5.169212053030185e-74),
  ((0, 2, 1), 4.396215399074331e-74),
  ((1,), 0.5816550235840346),
  ((1, 0, 2), 1.049079060904447e-73),
  ((1, 2), 0.22607603321880362),
  ((1, 2, 0), 1.454721947440344e-72),
  ((2, 0, 1), 7.66492634482424e-74),
  ((2, 1), 0.19226894319716173),
  ((2, 1, 0), 1.2497557492710257e-72)},
 {((0, 1, 2), 3.8910188386666

In [17]:
RPS_w[3]

{((0, 1, 2), 3.891018838666681e-27),
 ((0, 2, 1), 2.3087813150525835e-27),
 ((1,), 0.9176022547914116),
 ((1, 0, 2), 6.637943618793252e-27),
 ((1, 2), 0.051713147346844665),
 ((1, 2, 0), 1.1562906044195511e-26),
 ((2, 0, 1), 2.7440209721440796e-27),
 ((2, 1), 0.030684597861743695),
 ((2, 1, 0), 8.055665013028123e-27)}

## RPSR rule of combination


### Step 1, 2
Set fusion order and reliability vector   
**Default is descending order**

In [18]:
default_fusion_order = [i for i in range(num_attributes)]
default_reliability_vector = [(1 - 0.5 * i / (num_attributes - 1)) for i in range(num_attributes)]
# default_reliability_vector = [(1 - i / (12)) for i in range(num_attributes)]
print("Default fusion order: ", default_fusion_order)
print("Default reliability vector: ", default_reliability_vector)

Default fusion order:  [0, 1, 2, 3]
Default reliability vector:  [1.0, 0.8333333333333334, 0.6666666666666667, 0.5]


In [19]:
def shuffle_by_defusion_order(data, order):
    reordered_data = [data[i] for i in order]
    return reordered_data

RPS_wv = shuffle_by_defusion_order(RPS_w, default_fusion_order)
RPS_wv

[{((0, 1, 2), 1.5917244981330598e-07),
  ((0, 2, 1), 2.9406750290725125e-07),
  ((1, 0, 2), 2.0213860437974062e-07),
  ((1, 2), 0.09118749040651578),
  ((1, 2, 0), 9.460419907293513e-07),
  ((2,), 0.7403419597193839),
  ((2, 0, 1), 6.003605197431245e-07),
  ((2, 1), 0.16846682721585785),
  ((2, 1, 0), 1.5208771748850276e-06)},
 {((0, 1, 2), 0.01188199867148949),
  ((0, 2, 1), 0.01035662243762494),
  ((1,), 0.47885110874073056),
  ((1, 0, 2), 0.015796363493213478),
  ((1, 2), 0.22509177919736081),
  ((1, 2, 0), 0.025677447776139048),
  ((2, 0, 1), 0.012617876180362793),
  ((2, 1), 0.19619515499138454),
  ((2, 1, 0), 0.02353164851169426)},
 {((0, 1, 2), 5.169212053030185e-74),
  ((0, 2, 1), 4.396215399074331e-74),
  ((1,), 0.5816550235840346),
  ((1, 0, 2), 1.049079060904447e-73),
  ((1, 2), 0.22607603321880362),
  ((1, 2, 0), 1.454721947440344e-72),
  ((2, 0, 1), 7.66492634482424e-74),
  ((2, 1), 0.19226894319716173),
  ((2, 1, 0), 1.2497557492710257e-72)},
 {((0, 1, 2), 3.8910188386666

### Step 3
For each RPS source RPSjv, produce RPS with reliability RPSjrv

In [20]:
# Function to generate all permutations and combinations
def generate_all_orderings(num):
    result = []
    
    # Iterate over lengths from 1 to 3 (based on the provided examples)
    for length in range(1, 4):
        # Generate all permutations of the set {0, 1, 2} of the given length
        for perm in itertools.permutations(range(num), length):
            result.append(perm)
    
    return result

# Get the result
orderings = generate_all_orderings(3)

orderings


[(0,),
 (1,),
 (2,),
 (0, 1),
 (0, 2),
 (1, 0),
 (1, 2),
 (2, 0),
 (2, 1),
 (0, 1, 2),
 (0, 2, 1),
 (1, 0, 2),
 (1, 2, 0),
 (2, 0, 1),
 (2, 1, 0)]

In [21]:
def F_RPS_reliability(x):
    result = 0
    for k in range(x + 1):
        result += math.factorial(x) / math.factorial(x - k)
    return result

# def get_PMF_with_reliability(RPS_wv, reliability_vector, num_classes):
#     RPS_wv_r = []

#     for v, RPS_wv_j in enumerate(RPS_wv):
#         RPS_wv_r_j = set()
        
#         r_v = reliability_vector[v]
   
#         for A_tuple in RPS_wv_j:
#             A, MA = A_tuple
#             if len(A) == 1:
#                 MA_r = MA * r_v
#             elif len(A) != 1:
#                 MA_r = MA * r_v + ((1 - r_v) / (F_RPS_reliability(num_classes) - num_classes - 1))
#             RPS_wv_r_j.add((A, MA_r))
            
    
#         RPS_wv_r.append(RPS_wv_r_j)
        
#     return RPS_wv_r
def get_PMF_with_reliability(RPS_wv, reliability_vector, num_classes):
    RPS_wv_r = []

    for v, RPS_wv_j in enumerate(RPS_wv):
        RPS_wv_r_j = set()
        
        r_v = reliability_vector[v]

        for A_tuple in generate_all_orderings(num_classes):
            MA = 0
            
            if A_tuple in dict(RPS_wv_j):
                MA = dict(RPS_wv_j)[A_tuple]
            
            if len(A_tuple) == 1:
                MA_r = MA * r_v
            elif len(A_tuple) != 1:
                MA_r = MA * r_v + ((1 - r_v) / (F_RPS_reliability(num_classes) - num_classes - 1))
                
            RPS_wv_r_j.add((A_tuple, MA_r))

        RPS_wv_r.append(RPS_wv_r_j)
        
    return RPS_wv_r

RPS_wv_r = get_PMF_with_reliability(RPS_wv, default_reliability_vector, num_classes)
RPS_wv_r

[{((0,), 0.0),
  ((0, 1), 0.0),
  ((0, 1, 2), 1.5917244981330598e-07),
  ((0, 2), 0.0),
  ((0, 2, 1), 2.9406750290725125e-07),
  ((1,), 0.0),
  ((1, 0), 0.0),
  ((1, 0, 2), 2.0213860437974062e-07),
  ((1, 2), 0.09118749040651578),
  ((1, 2, 0), 9.460419907293513e-07),
  ((2,), 0.7403419597193839),
  ((2, 0), 0.0),
  ((2, 0, 1), 6.003605197431245e-07),
  ((2, 1), 0.16846682721585785),
  ((2, 1, 0), 1.5208771748850276e-06)},
 {((0,), 0.0),
  ((0, 1), 0.013888888888888886),
  ((0, 1, 2), 0.023790554448463463),
  ((0, 2), 0.013888888888888886),
  ((0, 2, 1), 0.022519407586909672),
  ((1,), 0.3990425906172755),
  ((1, 0), 0.013888888888888886),
  ((1, 0, 2), 0.027052525133233453),
  ((1, 2), 0.20146537155335625),
  ((1, 2, 0), 0.03528676203567143),
  ((2,), 0.0),
  ((2, 0), 0.013888888888888886),
  ((2, 0, 1), 0.02440378570585788),
  ((2, 1), 0.17738485138170934),
  ((2, 1, 0), 0.03349859598196744)},
 {((0,), 0.0),
  ((0, 1), 0.027777777777777773),
  ((0, 1, 2), 0.027777777777777773),
  ((0

In [22]:
F_RPS_reliability(num_classes) - num_classes - 1

12.0

### Step 4
Combine the K PMFs with reliability based on LOS or ROS

In [ ]:
def right_intersection(A, B):
    """
    右正交 (RI)，即 B 中去除不在 A 中的元素
    """
    return tuple(item for item in B if item in A)

def left_intersection(A, B):
    """
    左正交 (LI)，即 A 中去除不在 B 中的元素
    """
    return tuple(item for item in A if item in B)

In [ ]:
# 测试用例
test_cases = [
    # 第一行测试用例
    (("R",), ("G",), tuple(), tuple()),        # 应该返回空 (∅)
    (("R",), ("R", "B"), ("R",), ("R",)),    # (R) left and right intersections
    (("R",), ("R", "G", "B"), ("R",), ("R",)),   # (R) left and right intersections
    (("R",), ("G", "B", "R"), ("R",), ("R",)),   # (R) left and right intersections

    # 第二行测试用例
    (("R", "B"), ("G",), tuple(), tuple()),           # 空 (∅)
    (("R", "B"), ("R", "B"), ("R", "B"), ("R", "B")),  # (R, B) left and right intersections
    (("R", "B"), ("R", "G", "B"), ("R", "B"), ("R", "B")),  # (R, B) left and right intersections
    (("R", "B"), ("G", "B", "R"), ("R", "B"), ("B", "R")),  # (R, B) left and right intersections

    # 第三行测试用例
    (("B", "R"), ("G",), tuple(), tuple()),  # 空 (∅)
    (("B", "R"), ("R", "B"), ("B", "R"), ("R", "B")),  # (B, R) left and right intersections
    (("B", "R"), ("R", "G", "B"), ("B", "R"), ("R", "B")),  # (B, R) left and right intersections
    (("B", "R"), ("G", "B", "R"), ("B", "R"), ("B", "R")),  # (B, R) left and right intersections
]

# 运行测试
for i, (A, B, expected_li, expected_ri) in enumerate(test_cases):
    li_result = left_intersection(A, B)
    ri_result = right_intersection(A, B)

    print(f"Test case {i+1}:")
    print(f"Left Intersection (A: {A}, B: {B}): {li_result}, expected: {expected_li}")
    print(f"Right Intersection (A: {A}, B: {B}): {ri_result}, expected: {expected_ri}")
    print("="*50)

In [ ]:
def calculate_KR(M1, M2):
    """
    计算右正交和的 K^R (K_R)
    """
    K_R = 0
    for B, w1 in M1:
        for C, w2 in M2:
            if right_intersection(B, C) == ():
                K_R += w1 * w2
    return K_R

def calculate_KL(M1, M2):
    """
    计算左正交和的 K^L (K_L)
    """
    K_L = 0
    for B, w1 in M1:
        for C, w2 in M2:
            if left_intersection(B, C) == ():
                K_L += w1 * w2
    return K_L

def ROS(M1, M2):
    """
    右正交和 (ROS)
    """
    K_R = calculate_KR(M1, M2)
    result = set()

    if K_R != 1:  # 防止 K_R 为 1 时出现除以 0 的情况
        for A, w1 in M1:
            weight_sum = 0
            for B, w1 in M1:
                for C, w2 in M2:
                    if right_intersection(B, C) == A:
                        weight_sum += w1 * w2
            if weight_sum > 0:
                result.add((A, (1 / (1 - K_R)) * weight_sum))

    return result

def LOS(M1, M2):
    """
    左正交和 (LOS)
    """
    K_L = calculate_KL(M1, M2)
    result = set()

    if K_L != 1:  # 防止 K_L 为 1 时出现除以 0 的情况
        for A, w1 in M1:
            weight_sum = 0
            for B, w1 in M1:
                for C, w2 in M2:
                    if left_intersection(B, C) == A:
                        weight_sum += w1 * w2
            if weight_sum > 0:
                result.add((A, (1 / (1 - K_L)) * weight_sum))

    return result

In [ ]:
# 输入的数据结构 (集合)
M1 = {
    ((0,), 0.4),
    ((0, 2), 0.3),
    ((2, 0), 0.3)
}

M2 = {
    ((1,), 0.4),
    ((0, 2), 0.1),
    ((0, 1, 2), 0.15),
    ((1, 2, 0), 0.35)
}

# 计算右正交和 (ROS)
ros_result = ROS(M1, M2)
print("Right Orthogonal Sum (ROS):")
for item in ros_result:
    print(item)

# 计算左正交和 (LOS)
los_result = LOS(M1, M2)
print("\nLeft Orthogonal Sum (LOS):")
for item in los_result:
    print(item)

In [ ]:
# 连续右正交和
def continuous_right_orthogonal_sum(PMFs):
    """
    连续执行右正交和操作
    :param PMFs: 输入的PMF列表
    :return: 最终的右正交和结果
    """
    result = PMFs[0]
    for i in range(1, len(PMFs)):
        result = ROS(result, PMFs[i])
    return result

# 连续左正交和
def continuous_left_orthogonal_sum(PMFs):
    """
    连续执行左正交和操作
    :param PMFs: 输入的PMF列表
    :return: 最终的左正交和结果
    """
    result = PMFs[0]
    for i in range(1, len(PMFs)):
        result = LOS(result, PMFs[i])
    return result

In [ ]:
fused_PMF_desc = continuous_left_orthogonal_sum(RPS_wv_r)
# for i in fused_PMF_desc:
#     print(i)
fused_PMF_desc

## Ordered probability transformation

In [ ]:
def calculate_OPT(pmf_data, num_classes):
    # 初始化 P_OPT, 每个类的初始概率为0
    P_OPT = {i: 0 for i in range(num_classes)}
    
    # 遍历 PMF 数据，处理每个排列事件
    for A, mass in pmf_data:
        if len(A) == 1:
            # 单元素事件，直接加到对应的 P_OPT 中
            theta = A[0]
            P_OPT[theta] += mass
        else:
            # 多元素事件
            last_element = A[-1]
            for theta in A:
                if theta != last_element:
                    P_OPT[theta] += mass / (len(A) - 1)
    
    return P_OPT

# 计算 OPT 概率分布
P_OPT = calculate_OPT(fused_PMF_desc, num_classes)

# 输出结果
print("P_OPT(θ):")
for theta, prob in P_OPT.items():
    print(f"P_OPT({theta}) = {prob}")